In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from sklearn import preprocessing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import plotly as py
from plotly import tools
from plotly.offline import iplot
from plotly.subplots import make_subplots
import seaborn as sns
sns.set()
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import TimeDistributed
from keras.optimizers import RMSprop, SGD, Adam
from keras.initializers import he_normal, he_uniform
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import statsmodels.api as sm
import time
import warnings
import itertools
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Installing pmdarima for auto arima
!pip install pmdarima

In [ ]:
import pmdarima as pm

In [ ]:
data=pd.read_csv(r'../input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv',parse_dates=['created_at'])
print("Data Dimensions are: ", data.shape)
data.columns=data.columns.str.replace(" ", "_").str.lower()
print("Columns: ", data.columns)

In [ ]:
data.head()

In [ ]:
#Data exploration
data.info()

Let's check data types of columns

In [ ]:
data.dtypes

**Checking for missing values**

In [ ]:
# Quantifying null values
print(data.isnull().sum())

In [ ]:
# Visualizing the percentage of null values
data.isnull().mean().plot.bar(figsize=(12,6))
plt.ylabel('Percentage of missing values')
plt.xlabel('Variables')
plt.title('Quantifying missing data')

****Above visualization shows percentage of missing value for each column in dataset ****
1. Columns Unnamed 21 to Unnamed 25 are 100% null
2. Columns sales_commission_code has above 50% null values
3. All the remaining columns have about 40% null values

In [ ]:
data.drop(["unnamed:_21", "unnamed:_22", "unnamed:_23", "unnamed:_24", "unnamed:_25"], axis = 1, inplace=True)
data.dropna(how='all', axis=0, inplace=True)
data.rename(columns={"_mv_": "mv", "category_name_1": "category_name"}, inplace = True)

In [ ]:
# Quantifying null values
print(data.isnull().sum())

In [ ]:
# inspect unique values - categorical variable
data['status'].unique()

In [ ]:
data.groupby('bi_status')['status'].value_counts()

**From above observations it is concluded that all statuses falls under group Gross can be marked as Canceled, Net and Valid group of orders can be considered under complete category**

In [ ]:
data['status'] = data['status'].replace(r'\\N', 'Cancelled', regex=True)

In [ ]:
# For simplicity we can merge all statuses into Completed, Cancelled and Refund
dict_status = {'Completed':['complete','closed','received','cod','paid','exchange','payment_review','pending','processing','holded','pending_paypal'],'Refund':['order_refunded','refund'], 'Cancelled':['canceled','fraud',np.nan]}
for n in range(len(dict_status)):
    key,value=list(dict_status.items())[n]
    data['status'].replace(value, key,inplace=True)
    n+=1

In [ ]:
data['status'].value_counts()

In [ ]:
# bar plots for status categorical variables

data['status'].value_counts().plot.bar()
plt.xticks(rotation=0)
plt.ylabel('Count')
plt.title('Status  - Distinct Counts')

# **Let's check columns for duplicate entries, non relevant data**

In [ ]:
# Check for relevance of different features of dataset
data[['created_at','working_date','sku','qty_ordered','price','grand_total','mv','discount_amount','sales_commission_code','customer_id']].head()

****We can drop the columns working date, mv, sales_commission_code & other irrelevant cols because we have all the required relevant information in columns created_at, grand_total and discount_amount ****

In [ ]:
data.drop(['working_date', 'mv', 'increment_id','bi_status','sales_commission_code'],axis=1,inplace=True)

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

**Checking null/not defined values in categorical variable category_name**

In [ ]:
print("Count Different Categories: ")
print(data['category_name'].value_counts(dropna=False)) 

In [ ]:
# Extracting all unique categories for category value'\N'
skunique=data[data['category_name']==r'\N']['sku'].unique().tolist()

In [ ]:
#Now we check for sku's found against '\N' category in other categories 
sku_nil=data[data['sku'].isin(skunique)]
sku_nil['category_name'].value_counts()

In [ ]:
# We found sku with '\N' category also in categories as mentioned above
# Updating the sku category '\N' where same sku found in above categories and all remaining values of sku with Other's  
dict_sku={}
cat=["Men's Fashion",'Others','Superstore','Mobiles & Tablets',"Women's Fashion",'Entertainment','Appliances']
for n in cat:
    dict_sku[n]= n
for n in range(len(dict_sku)):
    key,value=list(dict_sku.items())[n]
    dict_sku[key]=sku_nil[sku_nil['category_name']== key]['sku'].unique().tolist()
    data.loc[((data['sku'].isin(dict_sku[key])) & (data['category_name']==r'\N')),'category_name']= key
    n+=1


In [ ]:
data.loc[(data['category_name']==r'\N'),'category_name']= 'Others'
data['category_name'] = data['category_name'].replace(np.nan, 'Others', regex=True)

In [ ]:
# Categories after updating all '\N' and null categories
data['category_name'].value_counts(dropna=False).plot.bar(figsize=(12,6))
#plt.xticks(rotation=0)
plt.ylabel('Sales')
plt.title('Sales Category Wise')

**Checking categorical variable sku for null entries**

In [ ]:
data[data['sku'].isnull()]

In [ ]:
# As out of 20 null sku values,  most of the order statuses are either cancelled or refund with grand_total 0.
# Replacing nan values with sku_nan
data['sku'].fillna("sku_nan",inplace=True)

In [ ]:
#Checking null values in columns customer_id and customer_since
data[data['customer_id'].isnull()]

In [ ]:
#For customer_id null most of the orders are with order status as cancelled or refund
#We can replace the null customer_id with value 0 and customer_since with 2018
data['customer_id'].fillna("0",inplace=True)
data['customer_since'].fillna("1-2018",inplace=True)

In [ ]:
#Replacing values < 0 with 0
data.loc[(data['grand_total']< 0), 'grand_total']=0

# **Time Series analysis and Order predictions**

In [ ]:
# Filter all competed orders 
dataordercomp= data[data['status']=='Completed']

In [ ]:
#Get total of daily sales
df=dataordercomp.groupby('created_at').size().reset_index(name='orders')
df.columns=['date','orders']
df.set_index('date',inplace=True)

In [ ]:
plt.rcParams["figure.figsize"] = [16,9]

In [ ]:
#Check general order trend
plt.plot(df.orders)


In [ ]:
df1=df
df1['year'] = df1.index.year
df1['month'] = df1.index.month
fig = px.box(df1, x="year", y="orders")
fig.update_layout(
    title_text='Yearly Trend'
    )
fig.show()

In [ ]:
fig = px.box(df1, x="month", y="orders")
fig.update_layout(
    title_text='Monthly Seasonality'
    )
fig.show()

In [ ]:
#For checking stationarity of series we use adfuller test
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
adf_test(df.orders)

In [ ]:
#Results shows that our data is not stationary, now we apply different techniques to make data stationary
#
#df['orders']=df.orders.rolling(window=5).mean()
df['orders']=np.log(df.orders)
#df['orders']=df.orders.diff(periods=5)#.dropna(inplace=True)#fillna(0)
df['orders']=df['orders'].ewm(alpha=0.5).mean()

Log transform makes the series stationary, whereas differencing is not effective for this dataset. Similarly used Exponential Weighted Moving Average for smoothing.

In [ ]:
adf_test(df.orders)

In [ ]:
df.index = pd.date_range(start='2016-07-01',periods=789,freq='D') 

In [ ]:
sm.graphics.tsa.plot_acf(df.orders);

In [ ]:
sm.graphics.tsa.plot_pacf(df.orders);

In [ ]:
# To filter best p,q and d parameters for ARIMA model we use autorima function
model = pm.auto_arima(df.orders, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(df.orders, order=(3,0,3))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
plt.plot(df.orders,color='r')
plt.plot(model_fit.predict(dynamic=False),color='g')
plt.legend(['Actual', 'Predicted'])

In [ ]:
# Checking model results
model_fit.plot_diagnostics(figsize=(15, 12))
plt.show()

Overall, the model fit seems good. Residual errors overall shows uniform variance, histogram shows distribituion is likerly normal and most of the theoratical quantities are perfectly in line with the red line.So, let's use it to forecast.

In [ ]:
pred = model_fit.get_prediction(start=pd.to_datetime('2018-08-1'), dynamic=True)
pred_ci = pred.conf_int()

In [ ]:
ax = df.loc['2018':]['orders'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='Forecasted', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Orders')
plt.legend()

plt.show()

In [ ]:
# Extract the predicted and true values of our time series
y_forecasted = pred.predicted_mean
y_truth = df.loc['2018-08-1':]['orders']

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

rmse =sqrt(mse)
print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse, 2)))

mape = np.mean(np.abs(np.exp(y_forecasted) - np.exp(y_truth))/np.abs(np.exp(y_truth))) # MAPE
print('The Mean Absolute Percentage Error of our forecasts is {}'.format(round(mape, 2)))

In [ ]:
# Get forecast 60 steps ahead in future
pred_uc = model_fit.get_forecast(steps=60)

# Get confidence intervals of forecasts
pred_ci = pred_uc.conf_int()

In [ ]:
ax = df.orders.plot(label='observed', figsize=(20, 15))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.09)
ax.set_xlabel('Date')
ax.set_ylabel('No of Orders')

plt.legend()
plt.title('ARIMA Model')
plt.show()

In [ ]:
# Seasonal - fit stepwise auto-ARIMA
smodel = pm.auto_arima(df.orders, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

smodel.summary()

In [ ]:
# Now applying sesonal arima 
mod = sm.tsa.statespace.SARIMAX(df['orders'],
                                            order=(2,0,0),
                                            seasonal_order=(2,1,0,12),
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
#mod = sm.tsa.statespace.SARIMAX(df['orders'], trend='c', order=(21,0,(1,0,0,1)))
res = mod.fit()
print(res.summary())

In [ ]:
res.plot_diagnostics();

Overall, the model fit seems good. Residual errors overall shows uniform variance, histogram shows distribituion is likerly normal and most of the theoratical quantities are perfectly in line with the red line.So, let's use it to forecast.

In [ ]:
pred = res.get_prediction(start=pd.to_datetime('2018-08-1'), dynamic=True)
pred_ci = pred.conf_int()

In [ ]:
ax = df.loc['2018':]['orders'].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='Forecasts', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Orders')
plt.legend()

plt.show()

In [ ]:
# Extract the predicted and true values of our time series
y_forecasted = pred.predicted_mean
y_truth = df.loc['2018-08-1':]['orders']

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

rmse =sqrt(mse)
print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse, 2)))

mape = np.mean(np.abs(np.exp(y_forecasted) - np.exp(y_truth))/np.abs(np.exp(y_truth))) # MAPE
print('The Mean Absolute Percentage Error of our forecasts is {}'.format(round(mape, 2)))

In [ ]:
# Get forecast 60 steps ahead in future
pred_uc = res.get_forecast(steps=60)

# Get confidence intervals of forecasts
pred_ci = pred_uc.conf_int()

In [ ]:
ax = df.orders.plot(label='observed', figsize=(20, 15))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=0.05)
ax.set_xlabel('Date')
ax.set_ylabel('Orders')

plt.legend()
plt.title('Seasonal ARIMA Model')
plt.show()

In this notebook classical time series forecasting models are used. In the next version I will try to explore LSTM.   